# Task 1: Binary Representations

In this section, It creates some basic bit manipulation functions.
Trying to rotate bits to the left

References from https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

Implements rotl(x,n), rotating left

In [13]:
# First try at rotate left... but I missed something

def rotl(x, n=1):
    return (x << n) | (x >> n)

# Testing with a simple number that only has 1 bit on
print(bin(rotl(0b00000000000000000000000000000001, 1)))


0b10


does not rotate correctly.
shifted right by n, but to wrap the bits properly I should shift right by 32 - n. Also, forgot to make sure we stay within 32 bits .

In [ ]:
# Fixed it by wrapping the bits properly and keeping the result to 32 bits.
# ref on https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

def rotatel(x, n=1):

    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# Testing again
print(bin(rotatel(0b00000000000000000000000000000001, 1)))  
print(bin(rotatel(0x80000000, 1)))  


0b10
0b1


rotr (x,n-1) rotate right

this is basically the opposite of the last one.
rotating bits to the right, and wrapping it back to the left

ref : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

In [ ]:
# First attempt at rotating right – but something feels off

def rotater(x, n=1):
    return (x >> n) | (x << n)

print(bin(rotater(0b00000000000000000000000000000010, 1)))  # expecting 0b1


0b101


In [22]:
# Fixed it 
def rotater(x, n=1):
   
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

print(bin(rotater(0b10, 1)))   
print(bin(rotater(0b1, 1)))    


0b1
0b10000000000000000000000000000000



the `return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF` function spins the bits to the right by n steps making sure everything stays within 32 bits  see https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb sections: bitwise shift , bitwise OR , bit masking , integer size